In [1]:
# scrabble_ai_training.ipynb

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
import pickle
import random

# -------------------------------
# 1️⃣ Generate synthetic Scrabble-like data
# -------------------------------

letters = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

def generate_board_features():
    """
    Simulate board as numerical features:
    - letter count
    - vowel count
    - consonant count
    - average letter value (A=1, Z=26)
    """
    board_letters = random.choices(letters, k=random.randint(5, 20))
    counts = len(board_letters)
    vowels = sum(ch in "AEIOU" for ch in board_letters)
    consonants = counts - vowels
    avg_value = np.mean([ord(ch) - ord('A') + 1 for ch in board_letters])
    return [counts, vowels, consonants, avg_value]

def estimate_score(features):
    """
    Synthetic target function — assume more vowels & higher avg value → better move.
    """
    counts, vowels, consonants, avg_value = features
    return 0.2 * counts + 0.5 * vowels + 0.3 * avg_value + random.uniform(0, 5)

# Generate training data
N_SAMPLES = 2000
X = np.array([generate_board_features() for _ in range(N_SAMPLES)])
y = np.array([estimate_score(f) for f in X])

# Normalize features
X = (X - X.mean(axis=0)) / X.std(axis=0)
y = (y - y.min()) / (y.max() - y.min())

print("✅ Data shape:", X.shape, y.shape)

# -------------------------------
# 2️⃣ Define a simple feed-forward model (no CNN)
# -------------------------------

model = Sequential([
    Dense(32, activation='relu', input_shape=(4,)),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')  # Predict move score
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
model.summary()

# -------------------------------
# 3️⃣ Train model
# -------------------------------

history = model.fit(X, y, epochs=20, batch_size=16, validation_split=0.1)
print("✅ Training complete.")

# -------------------------------
# 4️⃣ Save model + tokenizer
# -------------------------------

model.save("model/model_weights.h5")
print("✅ Model saved to model/model_weights.h5")

tokenizer = {ch: i+1 for i, ch in enumerate(letters)}
with open("model/tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)
print("✅ Tokenizer saved.")


✅ Data shape: (2000, 4) (2000,)


D:\anaconda\envs\scrabble-ai\lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 32)                  │             160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           1,056 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,249 (4.88 KB)

 Trainable params: 1,249 (4.88 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
113/113 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.1373 - mae: 0.2759 - val_loss: 0.0227 - val_mae: 0.1246
Epoch 2/20
113/113 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0301 - mae: 0.1385 - val_loss: 0.0177 - val_mae: 0.1074
Epoch 3/20
113/113 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0202 - mae: 0.1153 - val_loss: 0.0135 - val_mae: 0.0947
Epoch 4/20
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0158 - mae: 0.1018 - val_loss: 0.0126 - val_mae: 0.0920
Epoch 5/20
113/113 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0144 - mae: 0.0986 - val_loss: 0.0117 - val_mae: 0.0907
Epoch 6/20
113/113 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0130 - mae: 0.0954 - val_loss: 0.0116 - val_mae: 0.0898
Epoch 7/20
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0128 - mae: 0.0936 - val_loss: 0.0114 - val_mae: 0.0904
Epoch 8/20
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0123 - mae: 0.0927 - val_loss: 0.0114 - val_mae: 0.0908
Epoch 9/20
113/113 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - lo

✅ Training complete.
✅ Model saved to model/model_weights.h5
✅ Tokenizer saved.
